In [1]:
import pandas as pd
from web3 import Web3
import json
import time

# Load IoT dataset generated from Homework 1
df = pd.read_csv("iot_data.csv")
df.head()

,timestamp,package_id,order_date,delivery_date,origin,origin_coordinates,current_location,current_coordinates,delivery_location,delivery_coordinates,perishable,temperature_celsius,temperature_issue,status
0,2025-05-09 00:07:54,PKG445,2025-05-08,2025-05-14,"Valenzuela, Metro Manila","14.691690,120.969453","Valenzuela, Metro Manila","14.691690,120.969453","Tabuk, Kalinga","17.413619,121.444014",Yes,11.54°C,Temp Alert >7°C,To Ship
1,2025-05-09 00:25:17,PKG334,2025-05-06,2025-05-17,"Puerto Princesa, Palawan","9.780878,118.730191","Busuanga, Palawan","12.451537,120.089177","Antipolo, Rizal","14.587197,121.175925",No,26.1°C,NaN,In Transit
2,2025-05-09 00:41:22,PKG105,2025-05-02,2025-05-03,"Tarlac City, Tarlac","15.486122,120.589347","San Carlos, Pangasinan","15.926773,120.347558","San Carlos, Pangasinan","15.926773,120.347558",Yes,7.44°C,Temp Alert >7°C,Delivered
3,2025-05-09 01:31:24,PKG061,2025-05-01,2025-05-02,"Angeles, Pampanga","15.051732,120.700357","Gapan, Nueva Ecija","15.312281,120.947943","Gapan, Nueva Ecija","15.312281,120.947943",No,12.1°C,NaN,Delivered
4,2025-05-09 01:45:47,PKG297,2025-05-05,2025-05-16,"Cotabato City, Maguindanao del Norte","7.223763,124.246706","Tapaz, Capiz","11.268350,122.427083","Las Piñas, Metro Manila","14.480901,120.981786",No,27.53°C,NaN,In Transit


In [2]:
# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("Connected to Ganache successfully.")
else:
    raise ConnectionError("Connection failed. Ensure Ganache is running.")

Connected to Ganache successfully.


In [3]:
# Load the ABI and contract address from Remix
with open("abi.json", "r") as abi_file:
    abi = json.load(abi_file)

contract_address = web3.to_checksum_address("0xa9109dC13Ef7664b4edbBd1964F62f82ece5D98f")
contract = web3.eth.contract(address=contract_address, abi=abi)

In [4]:
# Set default sender (typically the first account from Ganache)
web3.eth.default_account = web3.eth.accounts[0]
print(f"Connected to Smart Contract at {contract_address}")

Connected to Smart Contract at 0xa9109dC13Ef7664b4edbBd1964F62f82ece5D98f


In [5]:
# Define a function to send each row of IoT data to the blockchain
def send_iot_data(row):
    txn = contract.functions.storeData(
        str(row["package_id"]),
        str(row["order_date"]),
        str(row["delivery_date"]),
        str(row["origin"]),
        str(row["current_location"]),
        str(row["delivery_location"]),
        str(row["perishable"]),
        str(row["temperature_celsius"]),
        str(row["temperature_issue"]),
        str(row["status"])
    ).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"Stored {row['package_id']} | Status: {row['status']} | Txn Hash: {receipt.transactionHash.hex()}")

In [6]:
# Loop through the DataFrame and send each row to the blockchain
for _, row in df.iterrows():
    send_iot_data(row)
    time.sleep(1)  # Avoid flooding the blockchain with transactions

Stored PKG445 | Status: To Ship | Txn Hash: e37f8d635c48996a2caf6ede5a3d50b3d50c64eb6438cc1dcb1f6e296e3c6f90
Stored PKG334 | Status: In Transit | Txn Hash: ad3c87d4e1052fc56017251a32b26a4431b20b0f0ab9d5fd3129c24e09bf7421
Stored PKG105 | Status: Delivered | Txn Hash: e4bf5a44ee44c8a9e8b1c433f1cdd687147766d78231f638440786b39f15efe1
Stored PKG061 | Status: Delivered | Txn Hash: 4b0ab9e1b7101f52159f2f36d9af86903f45f768a3c9674f03cf2c5bff2cb4f0
Stored PKG297 | Status: In Transit | Txn Hash: c210460716059704e60905a956dd61d5ccb515a7358818275b3b70533703689e
Stored PKG155 | Status: In Transit | Txn Hash: 3664a796a336eab489d68c94c098fee282f10c88ec2ad768c614bcb6ee88e3c8
Stored PKG141 | Status: In Transit | Txn Hash: f547d8ff08e390c4d20388e9b3c6c9e27b5fea095d6546e910f22fc0d8631c9d
Stored PKG399 | Status: To Ship | Txn Hash: 75592a191742f2bbce76a21d1dae7f0e7c9b90b7539e4b7417af4be7a4b5e0a0
Stored PKG301 | Status: In Transit | Txn Hash: 1508afe714753250772eea205e5b1d3c4463d8d4adee4b9cd9805dd83e976e11


In [7]:
# Verify the total number of records stored
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

Total IoT records stored: 500


In [8]:
# Retrieve and display the first stored record
first_record = contract.functions.getRecord(0).call()
print("First Stored Record:", first_record)

First Stored Record: [1748768335, 'PKG445', '2025-05-08', '2025-05-14', 'Valenzuela, Metro Manila', 'Valenzuela, Metro Manila', 'Tabuk, Kalinga', 'Yes', '11.54°C', 'Temp Alert >7°C', 'To Ship']
